# Reporte multiples files sin observaciones

importar librerias

In [5]:
import pandas as pd
import os
import datetime

obtener ruta de archivos

In [6]:
base_dir = os.path.dirname(os.path.realpath('__file__'))
descargas = os.path.join(base_dir, 'descargas')
listado = os.listdir(descargas)
output = os.path.join(base_dir, 'output')
existe = os.path.exists(output)
if not existe:
    os.makedirs(output)    

iterar

In [7]:
for l in listado:
    files = os.path.join(descargas, l)
    data_s = pd.read_excel(files)
    
    nombre = data_s['Nombre'][1]    
    col_fh = data_s['Fecha/Hora']
    
    temp = {}
    main_dict = {}

    #crear encabezado y asignar una lista vacia
    cols = ['fecha', 'entrada', 'salida', 'hs_trab', 'observaciones']
    for i in range(len(cols)):
                main_dict[cols[i]] = []
    
    #contar si marca 1 o 2 veces
    cont = {}
    for r in col_fh:
        fecha, _ = str(r).split(' ')
        if fecha in cont:
            cont[fecha] = 2
        else:
            cont[fecha] = 1
    
    #si marco una vez agregar otra marca
    for i, j in cont.items():     
        if j == 1:     
            i += ' --'
            s1 = pd.Series(i)            
            col_fh =  col_fh.append(s1)
            
    #recorrer la columna fecha-hora
    for row in col_fh:
        fecha, hora = str(row).split(' ')
        try:
            hora = datetime.datetime.strptime(hora, '%H:%M:%S')
        except:
            hora = 'None'
                
        if fecha not in temp:
            temp[fecha] = []
            temp[fecha].append(fecha)
            temp[fecha].append(hora)
        else:
            temp[fecha].append(hora)
            fecha = temp[fecha][0]
            entrada = str(temp[fecha][1])[11:16]
            salida = str(temp[fecha][2])[11:16]              
            try:
                horas_t = str(temp[fecha][2] - temp[fecha][1])[:4]
            except:
                horas_t = 'None'     
            obs= '-' 

            fila = [fecha, entrada, salida , horas_t, obs]

            for i in range(len(cols)):
                main_dict[cols[i]].append(fila[i])   
    filename, _ = l.split('.')
    df = pd.DataFrame(main_dict)
    df = df.sort_values(by=['fecha'])
    df.to_excel(f"{output}\{filename}.xlsx", sheet_name=nombre, 
                columns=['fecha', 'entrada' ,'salida','hs_trab','observaciones'], 
                index=False) 
    